In [1]:
# packages
import os
import posixpath
import numpy as np
import pandas as pd
import geopandas as gpd
import datetime
import shapely

# # warnings
# import warnings
# warnings.filterwarnings("ignore")

## Rename and organise Landsat fronts

In [2]:
data_root = "D:/OneDrive/Documents/Cours/4A/SFE/data/"
landsat_fronts_root = posixpath.join(data_root, "Landsat_MSS_fronts_old")
landsat_fronts = gpd.read_file(os.path.join(landsat_fronts_root, "Landsat_MSS_fronts_1.shp")).drop(columns=["date"])
landsat_fronts.columns = ["image", "mission", "timestamp", "RGI60ID", "geometry"]
landsat_fronts.head()

,image,mission,timestamp,RGI60ID,geometry
0,LM02_L1GS_245002_19760505_20200907_02_T2,Landsat 2 MSS,1976-05-05,RGI60-07.00791,"LINESTRING (12.38191 79.53251, 12.38144 79.531..."
1,LM02_L1GS_245002_19760505_20200907_02_T2,Landsat 2 MSS,1976-05-05,RGI60-07.00777,"LINESTRING (12.38935 79.54895, 12.38853 79.549..."
2,LM02_L1GS_245002_19760505_20200907_02_T2,Landsat 2 MSS,1976-05-05,RGI60-07.00775,"LINESTRING (12.32881 79.58411, 12.32937 79.585..."
3,LM02_L1GS_245002_19760505_20200907_02_T2,Landsat 2 MSS,1976-05-05,RGI60-07.00794,"LINESTRING (12.29071 79.65163, 12.28778 79.651..."
4,LM02_L1GS_245002_19760505_20200907_02_T2,Landsat 2 MSS,1976-05-05,RGI60-07.00790,"LINESTRING (12.1169 79.65285, 12.11207 79.6529..."


In [5]:
# add RGIv7 indexes
RGI_link = pd.read_csv(posixpath.join(data_root, "Randolph_glacier_inventory/links_v6_v7_svalbard.csv"))
landsat_fronts.loc[:, "RGI70ID"] = landsat_fronts["RGI60ID"].map(lambda x: RGI_link.loc[RGI_link.RGI60ID ==x].RGI70ID.values[0] if len(RGI_link.loc[RGI_link.RGI60ID ==x].RGI70ID.values) > 0 else None)
landsat_fronts = landsat_fronts.loc[:, ["image", "mission", "timestamp", "RGI60ID", "RGI70ID", "geometry"]]
landsat_fronts.head()

,image,mission,timestamp,RGI60ID,RGI70ID,geometry
0,LM02_L1GS_245002_19760505_20200907_02_T2,Landsat 2 MSS,1976-05-05,RGI60-07.00791,RGI2000-v7.0-G-07-01223,"LINESTRING (12.38191 79.53251, 12.38144 79.531..."
1,LM02_L1GS_245002_19760505_20200907_02_T2,Landsat 2 MSS,1976-05-05,RGI60-07.00777,RGI2000-v7.0-G-07-01225,"LINESTRING (12.38935 79.54895, 12.38853 79.549..."
2,LM02_L1GS_245002_19760505_20200907_02_T2,Landsat 2 MSS,1976-05-05,RGI60-07.00775,RGI2000-v7.0-G-07-01229,"LINESTRING (12.32881 79.58411, 12.32937 79.585..."
3,LM02_L1GS_245002_19760505_20200907_02_T2,Landsat 2 MSS,1976-05-05,RGI60-07.00794,RGI2000-v7.0-G-07-01268,"LINESTRING (12.29071 79.65163, 12.28778 79.651..."
4,LM02_L1GS_245002_19760505_20200907_02_T2,Landsat 2 MSS,1976-05-05,RGI60-07.00790,RGI2000-v7.0-G-07-01266,"LINESTRING (12.1169 79.65285, 12.11207 79.6529..."


In [23]:
landsat_fronts.dropna(inplace=True) # ok after manual inspection

In [ ]:
# /!\ Careful not to overwrite the original file

# landsat_fronts.to_file(posixpath.join(data_root, "Landsat_MSS_fronts/Landsat_MSS_fronts.shp"), driver="ESRI Shapefile")

PermissionError: [WinError 32] Le processus ne peut pas accéder au fichier car ce fichier est utilisé par un autre processus: 'D:/OneDrive/Documents/Cours/4A/SFE/data/Landsat_MSS_fronts/Landsat_MSS_fronts.shp'

## Rename and organise KH fronts

In [26]:
kh_fronts_root = posixpath.join(data_root, "KH-5_ARGON_fronts_old")
kh_fronts = gpd.read_file(os.path.join(kh_fronts_root, "KH-5_ARGON_fronts.shp")).drop(columns=["id"])
kh_fronts.head()

,image,RGI60ID,geometry
0,DS09058A024MC012,RGI60-07.00791,"LINESTRING (12.38466 79.53574, 12.38247 79.535..."
1,DS09058A024MC012,RGI60-07.00777,"LINESTRING (12.40558 79.54552, 12.4072 79.5460..."
2,DS09058A024MC012,RGI60-07.00775,"LINESTRING (12.33119 79.57981, 12.33606 79.580..."
3,DS09058A024MC012,RGI60-07.00709,"LINESTRING (10.90479 79.4142, 10.90802 79.4143..."
4,DS09058A024MC012,RGI60-07.00664,"LINESTRING (10.90592 79.36739, 10.90565 79.367..."


kh_fronts.image.value_counts()

In [27]:
def timestamp(image):
    if image == 'DS09034A008MC022':
        return datetime.datetime(1962, 5, 16) 
    if image == 'DS09034A008MC021':
        return datetime.datetime(1962, 5, 16) 
    if image == 'DS09058A024MC012':
        return datetime.datetime(1963, 8, 29) 
    if image == 'DS09058A024MC013':
        return datetime.datetime(1963, 8, 29) 
    if image == 'DS09034A008MC020':
        return datetime.datetime(1962, 5, 16)  
    if image == 'DS09034A007MC018':
        return datetime.datetime(1962, 5, 16) 
    if image == 'DS09034A007MC019':
        return datetime.datetime(1962, 5, 16)
    
kh_fronts.loc[:, "timestamp"] = kh_fronts.image.apply(timestamp)
kh_fronts.head()

,image,RGI60ID,geometry,timestamp
0,DS09058A024MC012,RGI60-07.00791,"LINESTRING (12.38466 79.53574, 12.38247 79.535...",1963-08-29
1,DS09058A024MC012,RGI60-07.00777,"LINESTRING (12.40558 79.54552, 12.4072 79.5460...",1963-08-29
2,DS09058A024MC012,RGI60-07.00775,"LINESTRING (12.33119 79.57981, 12.33606 79.580...",1963-08-29
3,DS09058A024MC012,RGI60-07.00709,"LINESTRING (10.90479 79.4142, 10.90802 79.4143...",1963-08-29
4,DS09058A024MC012,RGI60-07.00664,"LINESTRING (10.90592 79.36739, 10.90565 79.367...",1963-08-29


In [28]:
def mission(image):
    if image.startswith("DS09034"):
        return "KH-5 ARGON 9034A"
    if image.startswith("DS09058"):
        return "KH-5 ARGON 9058A"
    
kh_fronts.loc[:, "mission"] = kh_fronts.image.apply(mission)
kh_fronts.head()

,image,RGI60ID,geometry,timestamp,mission
0,DS09058A024MC012,RGI60-07.00791,"LINESTRING (12.38466 79.53574, 12.38247 79.535...",1963-08-29,KH-5 ARGON 9058A
1,DS09058A024MC012,RGI60-07.00777,"LINESTRING (12.40558 79.54552, 12.4072 79.5460...",1963-08-29,KH-5 ARGON 9058A
2,DS09058A024MC012,RGI60-07.00775,"LINESTRING (12.33119 79.57981, 12.33606 79.580...",1963-08-29,KH-5 ARGON 9058A
3,DS09058A024MC012,RGI60-07.00709,"LINESTRING (10.90479 79.4142, 10.90802 79.4143...",1963-08-29,KH-5 ARGON 9058A
4,DS09058A024MC012,RGI60-07.00664,"LINESTRING (10.90592 79.36739, 10.90565 79.367...",1963-08-29,KH-5 ARGON 9058A


In [29]:
kh_fronts = kh_fronts.loc[:, ["image", "mission", "timestamp", "RGI60ID", "geometry"]]
kh_fronts.head()

,image,mission,timestamp,RGI60ID,geometry
0,DS09058A024MC012,KH-5 ARGON 9058A,1963-08-29,RGI60-07.00791,"LINESTRING (12.38466 79.53574, 12.38247 79.535..."
1,DS09058A024MC012,KH-5 ARGON 9058A,1963-08-29,RGI60-07.00777,"LINESTRING (12.40558 79.54552, 12.4072 79.5460..."
2,DS09058A024MC012,KH-5 ARGON 9058A,1963-08-29,RGI60-07.00775,"LINESTRING (12.33119 79.57981, 12.33606 79.580..."
3,DS09058A024MC012,KH-5 ARGON 9058A,1963-08-29,RGI60-07.00709,"LINESTRING (10.90479 79.4142, 10.90802 79.4143..."
4,DS09058A024MC012,KH-5 ARGON 9058A,1963-08-29,RGI60-07.00664,"LINESTRING (10.90592 79.36739, 10.90565 79.367..."


In [30]:
# add RGIv7 indexes
kh_fronts.loc[:, "RGI70ID"] = kh_fronts["RGI60ID"].map(lambda x: RGI_link.loc[RGI_link.RGI60ID ==x].RGI70ID.values[0] if len(RGI_link.loc[RGI_link.RGI60ID ==x].RGI70ID.values) > 0 else None)
kh_fronts = kh_fronts.loc[:, ["image", "mission", "timestamp", "RGI60ID", "RGI70ID", "geometry"]]
kh_fronts.head()

,image,mission,timestamp,RGI60ID,RGI70ID,geometry
0,DS09058A024MC012,KH-5 ARGON 9058A,1963-08-29,RGI60-07.00791,RGI2000-v7.0-G-07-01223,"LINESTRING (12.38466 79.53574, 12.38247 79.535..."
1,DS09058A024MC012,KH-5 ARGON 9058A,1963-08-29,RGI60-07.00777,RGI2000-v7.0-G-07-01225,"LINESTRING (12.40558 79.54552, 12.4072 79.5460..."
2,DS09058A024MC012,KH-5 ARGON 9058A,1963-08-29,RGI60-07.00775,RGI2000-v7.0-G-07-01229,"LINESTRING (12.33119 79.57981, 12.33606 79.580..."
3,DS09058A024MC012,KH-5 ARGON 9058A,1963-08-29,RGI60-07.00709,RGI2000-v7.0-G-07-00011,"LINESTRING (10.90479 79.4142, 10.90802 79.4143..."
4,DS09058A024MC012,KH-5 ARGON 9058A,1963-08-29,RGI60-07.00664,RGI2000-v7.0-G-07-00015,"LINESTRING (10.90592 79.36739, 10.90565 79.367..."


In [31]:
kh_fronts.RGI70ID.isna().value_counts()

RGI70ID
False    518
Name: count, dtype: int64

In [ ]:
# /!\ Careful not to overwrite the original file

# kh_fronts.to_file(posixpath.join(data_root, "KH-5_ARGON_fronts/KH-5_ARGON_fronts.shp"), driver="ESRI Shapefile")

C:\Users\loris\AppData\Roaming\Python\Python312\site-packages\pyogrio\raw.py:723: RuntimeWarning: Field timestamp create as date field, though DateTime requested.
  ogr_write(
